In [ ]:
!pip uninstall -y kaggle
!pip install kaggle==1.5.6
!kaggle competitions download -c fcis-sc-deeplearning-competition
!unzip /content/fcis-sc-deeplearning-competition.zip

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!unzip  '/content/drive/My Drive/test.zip'

In [ ]:
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import keras
import os
import pandas as pd
import tqdm
from keras.preprocessing.image import ImageDataGenerator
from keras import Model, layers
from keras.models import load_model, model_from_json
from keras.applications.nasnet import NASNetLarge
from keras.applications.nasnet import preprocess_input
from keras.preprocessing import image
import PIL


TEST_DIR = '/content/test/test'
TRAIN_DIR = '/content/train/train'


train_datagen = ImageDataGenerator(
    shear_range=10,
    zoom_range=0.2,
    horizontal_flip=True,
    preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    batch_size=32,
    class_mode='categorical',
    target_size=(331,331))

conv_base = NASNetLarge(
    include_top=False,
    weights='imagenet')
for layer in conv_base.layers:
    layer.trainable = False
x = conv_base.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(128, activation='relu')(x) 
predictions = layers.Dense(10, activation='softmax')(x)
model = Model(conv_base.input, predictions)

optimizer = keras.optimizers.Adam()
from keras.optimizers import SGD
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
step_size_train = train_generator.n//train_generator.batch_size

model.fit_generator(train_generator,samples_per_epoch = step_size_train, epochs = 46)
model.save('NAS')
testing_pre = []

for imagep in os.listdir(TEST_DIR):
    image_path = os.path.join(TEST_DIR, imagep)
    img = image.load_img(image_path, target_size=(331,331))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis = 0)
    x = preprocess_input(x)
    preds = model.predict(x)
    #print(imagep, preds, np.argmax(preds) + 1)
    testing_pre.append([imagep,np.argmax(preds) + 1])

Csv = pd.DataFrame(testing_pre)
Csv.to_csv("subm1.csv")


print('end')

Found 3138 images belonging to 10 classes.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:50: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras_pre..., epochs=46, steps_per_epoch=3)`


Epoch 1/46
3/3 [==============================] - 25s 8s/step - loss: 2.1018 - acc: 0.2812
Epoch 2/46
3/3 [==============================] - 2s 578ms/step - loss: 1.6741 - acc: 0.5521
Epoch 3/46
3/3 [==============================] - 1s 416ms/step - loss: 1.6280 - acc: 0.6044
Epoch 4/46
3/3 [==============================] - 2s 579ms/step - loss: 1.2086 - acc: 0.6771
Epoch 5/46
3/3 [==============================] - 2s 577ms/step - loss: 0.9449 - acc: 0.7396
Epoch 6/46
3/3 [==============================] - 2s 579ms/step - loss: 0.7259 - acc: 0.8125
Epoch 7/46
3/3 [==============================] - 2s 578ms/step - loss: 0.8696 - acc: 0.7083
Epoch 8/46
3/3 [==============================] - 2s 674ms/step - loss: 0.6131 - acc: 0.8229
Epoch 9/46
3/3 [==============================] - 3s 967ms/step - loss: 0.8970 - acc: 0.7292
Epoch 10/46
3/3 [==============================] - 3s 953ms/step - loss: 0.6350 - acc: 0.8125
Epoch 11/46
3/3 [==============================] - 3s 994ms/step - loss